In [1]:
import pandas as pd
import numpy as np

idx1 = pd.date_range("2020-01-01", "2020-01-10", freq="min")
idx2 = pd.date_range("2020-01-01", "2020-01-10", freq="min")
idx = pd.concat([idx1.to_series(), idx2.to_series()]).sort_index()
versions = np.random.randint(1,5,len(idx))
vals1 = np.random.uniform(0, 100, len(idx))
vals2 = np.random.uniform(0, 100, len(idx))

In [4]:
len(versions)

25922

In [5]:
df = pd.DataFrame(index=idx, columns=["values1", "values2"])
df["values1"], df["values2"] = vals1, vals2
df.head()

,values1,values2
2020-01-01 00:00:00,78.760705,52.777184
2020-01-01 00:00:00,79.530675,69.868044
2020-01-01 00:01:00,31.225034,24.489480
2020-01-01 00:01:00,36.523530,39.653527
2020-01-01 00:02:00,56.160539,70.148007


In [6]:
%%timeit
lista = dict()
for index, timestamp in enumerate(df.index):
    if index > 0:
        if df.iloc[index]["values1"] > df.iloc[index-1]["values2"]:
            lista[df.index[index]] = df.iloc[index][["values1", "values2"]]

12.3 s ± 829 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
lista

NameError: name 'lista' is not defined

In [7]:
prev_idx = df.index.shift(freq="-1min")
prev_vals2 = df["values2"].shift(1).fillna(-1)
prev_vals2


2020-01-01 00:00:00    -1.000000
2020-01-01 00:00:00    52.777184
2020-01-01 00:01:00    69.868044
2020-01-01 00:01:00    24.489480
2020-01-01 00:02:00    39.653527
                         ...    
2020-01-09 23:58:00    90.046258
2020-01-09 23:59:00    89.396062
2020-01-09 23:59:00    52.848374
2020-01-10 00:00:00    98.663033
2020-01-10 00:00:00    70.771429
Name: values2, Length: 25922, dtype: float64

In [9]:
%%timeit
res = np.where(((df.index == prev_idx) & (df["values1"].values < prev_vals2)),
        df["values1"].values, df["values2"].values)

449 µs ± 5.51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
res[res > 0]

array([57.49487431, 79.48133615, 60.36623463, ...,  7.15592885,
        4.50433933, 58.18880318])

In [ ]:
%%timeit

def compare(row):
    if row > 0:
        if row["values1"] > df.iloc[row.index]